In [1]:
import sys ; sys.path.insert(0, "..")
from pathlib import Path
from src.datasets.dbg import dBGDataset

In [2]:
dbg_dataset = dBGDataset(root=Path("../dataset-covid-subsample/"), k=3, 
                         files=["../data/subsample-clade_O.fasta","../data/subsample-clade_S.fasta"], format="fasta")

Processing...
subsample-clade_O: 1000it [00:22, 44.89it/s]
subsample-clade_S: 1000it [00:56, 17.75it/s]
Done!


In [3]:
dbg_dataset.idx2metadata

[Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Brazil/SP-FIOCRUZ-12631/2023|2023-10-09|2023-12-08', idx=0),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Germany/NW-RKI-I-1140254/2021|2021-12-30|2023-12-08', idx=1),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Germany/SH-RKI-I-1140215/2022|2022-01-09|2023-12-08', idx=2),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Israel/SMC-7049656/2022|2022-03-02|2022-03-23', idx=3),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Germany/SH-RKI-I-1139951/2022|2022-01-30|2023-12-08', idx=4),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Germany/SH-RKI-I-1139926/2022|2022-01-23|2023-12-08', idx=5),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Italy/CAM-IZSM-RD00256778/2021|2021-03-02|2021-03-23', idx=6),
 Seq(filename='../data/subsample-clade_O.fasta', seqid='hCoV-19/Italy/CAM-IZSM-RD025322D57/2021|2021-03-20|2021-03-23', idx=